# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,27.32,78,100,8.52,MU,1706642449
1,1,stanley,54.8680,-1.6985,4.74,82,20,3.09,GB,1706642077
2,2,pariaman,-0.6190,100.1200,27.03,83,100,1.77,ID,1706642449
3,3,margaret river,-33.9500,115.0667,14.75,87,2,4.44,AU,1706642450
4,4,yellowknife,62.4560,-114.3525,-9.71,92,100,6.17,CA,1706642078


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)


# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
weather_preference = city_data_df.loc[(city_data_df['Max Temp'] <= 24) & (city_data_df['Max Temp'] >= 23) & (city_data_df['Cloudiness'] >= 40) & (city_data_df['Humidity'] >= 45)]
                 

# Drop any rows with null values
weather_preference = weather_preference.dropna()

# Display sample data
weather_preference.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,wailua homesteads,22.0669,-159.3780,23.24,82,40,1.79,US,1706642452
30,30,port elizabeth,-33.9180,25.5701,23.51,80,40,1.03,ZA,1706642457
36,36,adamstown,-25.0660,-130.1015,23.07,94,100,10.47,PN,1706642459
112,112,lihue,21.9789,-159.3672,23.58,82,40,0.00,US,1706642432
138,138,tsiombe,-25.3000,45.4833,23.05,77,100,7.62,MG,1706642497


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_preference[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel = '')
hotel_df.rename(columns={'Hotel': "Hotel Name"}, inplace=True)
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,wailua homesteads,US,22.0669,-159.3780,82,
30,port elizabeth,ZA,-33.9180,25.5701,80,
36,adamstown,PN,-25.0660,-130.1015,94,
112,lihue,US,21.9789,-159.3672,82,
138,tsiombe,MG,-25.3000,45.4833,77,
161,la'ie,US,21.6477,-157.9253,96,
166,port saint john's,ZA,-31.6229,29.5448,90,
213,kibara,TZ,-2.1500,33.4500,72,
280,langsa,ID,4.4683,97.9683,95,
465,palangkaraya,ID,-2.2000,113.8333,98,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
# Set the parameters for the type of place
categories = "accommodation.hotel"
limit = 20

# Set the parameters for the type of search

# set up a parameters dictionary
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.at[index,'Lat']
    longitude = hotel_df.at[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f'circle:{longitude},{latitude},{radius}'
    bias = f"proximity:{longitude},{latitude}"
    
    params["filter"] = filters
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
port elizabeth - nearest hotel: Waterford Hotel
adamstown - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
tsiombe - nearest hotel: No hotel found
la'ie - nearest hotel: No hotel found
port saint john's - nearest hotel: Outback Inn
kibara - nearest hotel: No hotel found
langsa - nearest hotel: Harmoni Hotel
palangkaraya - nearest hotel: Swiss-Belhotel
ca mau - nearest hotel: Khách sạn Song Ngọc
bandar lampung - nearest hotel: Grand Anugerah


,City,Country,Lat,Lng,Humidity,Hotel Name
11,wailua homesteads,US,22.0669,-159.3780,82,Hilton Garden Inn Kauai Wailua Bay
30,port elizabeth,ZA,-33.9180,25.5701,80,Waterford Hotel
36,adamstown,PN,-25.0660,-130.1015,94,No hotel found
112,lihue,US,21.9789,-159.3672,82,Kauai Palms
138,tsiombe,MG,-25.3000,45.4833,77,No hotel found
161,la'ie,US,21.6477,-157.9253,96,No hotel found
166,port saint john's,ZA,-31.6229,29.5448,90,Outback Inn
213,kibara,TZ,-2.1500,33.4500,72,No hotel found
280,langsa,ID,4.4683,97.9683,95,Harmoni Hotel
465,palangkaraya,ID,-2.2000,113.8333,98,Swiss-Belhotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
humidity_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols=['Country', 'Hotel Name']
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)